# TextRank 알고리즘 적용 위한 스텝
1. 텍스트 수집 (완)
2. 전처리
  - 불용어 처리 (re 사용)
3. tf-idf 통해 문서 내 특정 단어의 중요도 분석
  - 텍스트를 문장 단위로 분리
  - koNLPY twitter 활용해 형태소 태깅 + 조사 및 어간 분리 통해 명사 반환
  - TfidfVectorizer()와 CountVectorizer() 활용하여 tf, df 계산
    - 이때 word2vec, doc2vec 등을 결합하여 문서 간 유사도 계산 가능한 점 참고하기

4. tf-idf 과정을 통해 얻은 문장(단어) 중요도를 활용하여 ranking, 요약할 문장(단어) 수만큼 출력

공지 파일 결측치 41개 어떻게 처리할 것인가? --> 이미지의 손상으로 인해 데이터를 보완할 수 없기 때문에 삭제하기로 결정
- 만약 HUFS 공지 검색을 통해 보완할 수 있다면 최대한 보완할 것인가? 논의하기
- 보완 +로그 남기기 + 더이상 보완할 수 없을 경우 '직접 접속해서 확인해주세요.' 출력하기

2024.04.19
- 공지 파일 결측치 중 보완 가능한 23개 내용 보충 완료 후 재업로드
- 학사 인덱스 보충완료

### 할 순서(0425 회의 이후~)
1. 전번 등 임의 전처리 해서 데이터 전처리
- copy 시 코로나 관련 공지 빼기 (문법이 파괴되지 않는 한에서 한글 표현만 최대한 남기기)
2. kospacing 적용 _ 띄어쓰기 적용
3. 형태소 분석기 (Okt())로 형태소 단위 분리
3-1. 한글자 제거?
4. 불용어 리스트 가져와서 전처리(조사 등많이 제거될 것)
5. 빈도 확인후 전처리
- 최소이하 제거 or 빈도가 지나치게 높은 counter에 대해 임의 제거
- 잘못 분리된 단어 확인하여 사용자단어에 추가
6. TF-IDF 해보기
7. 전처리 반복

- 완벽한 전처리 결론이 목요일까지 나오는 것이 목표

- 참고한 전처리
https://haystar.tistory.com/11

- hanspell 관련확인 _clone도 안됨. 해결 코드가 있긴한데 일단 후순위로 미루고 작업
https://velog.io/@woalsgodqhr/%EB%84%A4%EC%9D%B4%EB%B2%84-%EB%A7%9E%EC%B6%A4%EB%B2%95-%EA%B2%80%EC%82%AC%EA%B8%B0-api-%EC%9C%A0%ED%9A%A8%ED%95%9C-%ED%82%A4%EA%B0%80-%EC%95%84%EB%8B%99%EB%8B%88%EB%8B%A4-%EC%98%A4%EB%A5%98-%ED%95%B4%EA%B2%B0%ED%95%98%EA%B8%B0

In [ ]:
# 공지 csv의 결측치 파악
# df2[df2['content'] == '\n\n']

In [ ]:
import pandas as pd
import os

In [ ]:
#경로확인
print(os.getcwd())

/content


In [ ]:
new_dir = '/content'

# 작업 디렉토리 변경
os.chdir(new_dir)

### 마운트 및 파일 로드부터 시작하기 (경로 설정)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#kospacing 받기
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
#작동확인
sent = '김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.'
new_sent = sent.replace(" ", '') # 띄어쓰기가 없는 문장 임의로 만들기
print(new_sent)

김철수는극중두인격의사나이이광수역을맡았다.철수는한국유일의태권도전승자를가리는결전의날을앞두고10년간함께훈련한사형인유연재(김광수분)를찾으러속세로내려온인물이다.


In [ ]:
from pykospacing import Spacing
spacing = Spacing()
kospacing_sent = spacing(new_sent)
#결과일치시 맞음
print(sent)
print(kospacing_sent)

In [ ]:
#결과일치시 맞음
print(sent)
print(kospacing_sent)

김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.
김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.


In [ ]:
gongji = pd.read_csv('./drive/MyDrive/csv모음/merge_gongji.csv')
haksa = pd.read_csv('./drive/MyDrive/csv모음/merged_haksa.csv')
hiring = pd.read_csv('./drive/MyDrive/csv모음/merged_hire.csv')
janghak = pd.read_csv('./drive/MyDrive/csv모음/merged_janghak.csv')
#공지가 너무 커 세개로 나눔
gongji_1 = pd.read_csv('./drive/MyDrive/csv모음/gongji_1.csv')
gongji_2 = pd.read_csv('./drive/MyDrive/csv모음/gongji_2.csv', encoding = "cp949")
gongji_3 = pd.read_csv('./drive/MyDrive/csv모음/gongji_3.csv')
gongji.head(5)

,index,date,title,content
0,1,2018-01-02,2018년 상반기 20주 주간/야간 외국어교육과정 교육생 모집,\n2018년 상반기 \n20주 \n주간/야간 외국어교육과정 교육생 \n모집기초부터...
1,2,2018-01-02,2018년 상반기 FLC 외국어교육과정 교육생 모집,\n2018년 상반기 FLC \n외국어교육과정 교육생 모집기초부터 고급까지! 해외연...
2,3,2018-01-02,2018년 상반기 10주 주말(토) 외국어교육과정 교육생 모집,\n2018년 상반기 \n10주 \n주말(토) 외국어교육과정 교육생 \n모집기초부터...
3,4,2018-01-03,대원빌딩 철거공사 입찰 관련 질의 응답,\n대원빌딩 철거공사 입찰 관련 질의 응답대원빌딩 철거공사 입찰과 관련한 질의사항에...
4,7,2018-01-04,2018학년도 제1학기 서울캠퍼스 국제학사 기숙사 신입생 모집 공고,\n2018학년도 제1학기 서울캠퍼스 국제학사 기숙사 신입생 모집 공고\n2018학...


# re 이용해 특정 문자열 제거

In [ ]:
# 원본 데이터 보존 위해 copy 데이터 생성
gongji_copy = gongji.copy()
haksa_copy = haksa.copy()
hiring_copy = hiring.copy()
janghak_copy = janghak.copy()


gongji1_copy = gongji_1.copy()
gongji2_copy = gongji_2.copy()
gongji3_copy = gongji_3.copy()

In [ ]:
haksa_copy.loc[5,'content']

'\n\n[서울] 2020년 전기 졸업예비사정 결과 안내\n\n서울캠퍼스 2020년 전기 졸업예비사정결과를 아래와 같이 공고함.\n\n아 래 \n\n1. 조회\n 종합정보시스템 -> 성적/학점/졸업관리 -> 졸업예비사정 결과 조회\n\n2. 기준 시점 : 2020. 1. 30기준\n\n3. 유의사항\n - 미비사항없음(졸업요건완료) 학생도 계절학기 재수강 과목 F로 졸업이 불가할 수 있음\n - 사범대/비사범대교직이수자 : 미비사항없음이 교원자격증 취득이 가능하다는 의미는 아님\n(교원자격증 취득 관련 : 사범대/교직 담당자에게 문의)\n -학과에서 제출한 졸업시험 및 논문, 단과대학 교학과에 제출한 외국어인증은 기준 시점까지\n모두 반영되었으므로 이상이 있을 경우 반드시 학과 및 단과대학 교학과 문의\n- 중국어대학의 경우 FLEX+졸업논문+학과졸업신청서의 이수내역 심사가 모두 합격하여야 반영되므로 논문만 제출한 경우 졸업시험/논문 내역이 반영되지 않음\n- 중국어대학의 경우 특기자여부 등을 파악하기 어려우므로 반드시 학과에 학과졸업신청서 작성시 이상유무를 확인하기 바람.\n- 재학연한초과제적예정이라고 표기된 학생은 이번에 졸업하지 않을 시 제적되는 학생이므로\n반드시 졸업요건을 완료하여 졸업하기 바람.\n- 미이수 내역에 전공필수미비 또는 이중전공필수미비 중 본인이 이상이 없다고 판단되는 경우 학과사무실 방문하여 확인을 받고 이상이 없는 경우 이상이 없음을 확인하는 서류를 받아 학사종합지원센터에 제출하여야 미비내역이 없어짐.\n\n\n1. 졸업에 필요한 미비조건 완료 \n2. 수료(졸업대기) 조건 완료\n *졸업학점만 완료하면 수료(졸업대기 )가능, 학점체크 하면 됨. 외국어인증이나 졸업(논문)시험 미비상태로 결과조회되면 됨\n\n 반드시 1.31(금) 오후 3시까지 미비 조건을 완료하여야 함.\n단, 외국어연수평가원 방학 특별과정 이수자는 해당 과정 종료일 당일\n수료증 제출자에한하여 졸업 또는 수료처리 가능.\n당일 미제출 시 졸업 또는 수료처리 불가.

### 전처리과정
- 전번, 주소, 메일 등 정보 부터 지우는 중
- 가독성 때문에 띄어쓰기 및 특수기호는 문자열 지운 이후에 마지막에 지울 듯 함
- 현재 test만 지웠고 데이터 프레임에 적용하지 않았음

In [ ]:
import re

In [ ]:
gongji_copy.loc[0,'content']

'\n2018년 상반기 \n20주 \n주간/야간 외국어교육과정 교육생 \n모집기초부터 고급까지! 해외연수 그 이상의 효과\n실용 회화능력 집중 향상! 국내 최고의 외국어마스터과정\n\n한국외국어대학교 외국어연수평가원\n2018년 상반기 20주 주간/야간 외국어교육과정 교육생 모집\n\n[과정소개]\n‘FLC 외국어교육과정’은 대학(원)생, 취업준비생, 해외연수 및 유학준비생, 인사혁신처 추천 공무원 및 일반 공무원, 기관 및 기업체 임직원, 회사원, 일반인 등을 대상으로 말하기, 듣기, 읽기, 쓰기의 체계적이고 강도높은 교육을 통해 실제 의사소통능력을 집중적으로 향상시켜 해외연수 그 이상의 효과를 거둘 수 있는 국내 최고의 외국어마스터 과정으로 특히, 과정 이수 후 교육성과 및 만족도 면에서 높은 평가로 프로그램의 우수성을 널리 인정받고 있습니다.\n\n1. 교육일정\n[20주 주간/야간 정규과정] “말하기, 듣기, 쓰기, 읽기의 균형있는 통합교육”\n구 분주 간야 간접수기간2017. 12. 11(월) 부터~교육기간2018. 02. 05(월)~06. 22(금)교육시간09:30~16:201일 6시간/월~금/주5일18:40~21:301일 3시간/월~금/주5일총 교육시수총 600시간총 300시간반편성시험(Level Test)2018. 01. 20(토) 예정(장소:외국어연수평가원)교 육 비3,737,000원(전형료 30,000원 포함, 교재비 별도)※재수강시 교육비 할인(별도문의)1,821,000원(전형료 30,000원 포함, 교재비 별도)※재수강시 교육비 할인(별도문의)※단, 위 교육비는 일반적인 기준의 교육비이며, 인사혁신처 추천 공무원 및 별도 위탁교육생의 경우 교육비가 다를 수 있으므로 별도 문의\n\n2. 교육언어\n- 영어, 중국어, 일본어, 스페인어, 베트남어, 프랑스어, 독일어, 러시아어, 마인어 외\n- 과정별 특성에 따라 모집언어는 다를 수 있으며 모집공지시 자세한 교육언어 참조\n- 언어/레벨별 적정인원 모집시 개설되며 모집인원 미달시에는 개설되지

In [ ]:
# 전화번호를 포함하는 패턴
tel_number = r'TEL\.\d{2}-?\d{3,4}-\d{4}'
#전번 양식 (번호만 있는 경우대비)
phone_number = r'\d{2}-\d{4}-\d{4}'
#팩스 양식
fax_number = r'FAX\.\d{2}-\d{3}-\d{4}'
#주소 양식
address = r'서울시 동대문구 이문로 107'
#이메일 양식
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
#링크 양식
link_http = r'http://\S+'
link_www = r'\bwww\.\S*'
link_hufs = r'hufs\.ac\.kr'

#문장부호 지우기
punctuation_pattern = r'[,.:!?\\|※‘’”“→-]'

In [ ]:
test = gongji_copy.loc[0,'content']
print(test)


2018년 상반기 
20주 
주간/야간 외국어교육과정 교육생 
모집기초부터 고급까지! 해외연수 그 이상의 효과
실용 회화능력 집중 향상! 국내 최고의 외국어마스터과정

한국외국어대학교 외국어연수평가원
2018년 상반기 20주 주간/야간 외국어교육과정 교육생 모집

[과정소개]
‘FLC 외국어교육과정’은 대학(원)생, 취업준비생, 해외연수 및 유학준비생, 인사혁신처 추천 공무원 및 일반 공무원, 기관 및 기업체 임직원, 회사원, 일반인 등을 대상으로 말하기, 듣기, 읽기, 쓰기의 체계적이고 강도높은 교육을 통해 실제 의사소통능력을 집중적으로 향상시켜 해외연수 그 이상의 효과를 거둘 수 있는 국내 최고의 외국어마스터 과정으로 특히, 과정 이수 후 교육성과 및 만족도 면에서 높은 평가로 프로그램의 우수성을 널리 인정받고 있습니다.

1. 교육일정
[20주 주간/야간 정규과정] “말하기, 듣기, 쓰기, 읽기의 균형있는 통합교육”
구 분주 간야 간접수기간2017. 12. 11(월) 부터~교육기간2018. 02. 05(월)~06. 22(금)교육시간09:30~16:201일 6시간/월~금/주5일18:40~21:301일 3시간/월~금/주5일총 교육시수총 600시간총 300시간반편성시험(Level Test)2018. 01. 20(토) 예정(장소:외국어연수평가원)교 육 비3,737,000원(전형료 30,000원 포함, 교재비 별도)※재수강시 교육비 할인(별도문의)1,821,000원(전형료 30,000원 포함, 교재비 별도)※재수강시 교육비 할인(별도문의)※단, 위 교육비는 일반적인 기준의 교육비이며, 인사혁신처 추천 공무원 및 별도 위탁교육생의 경우 교육비가 다를 수 있으므로 별도 문의

2. 교육언어
- 영어, 중국어, 일본어, 스페인어, 베트남어, 프랑스어, 독일어, 러시아어, 마인어 외
- 과정별 특성에 따라 모집언어는 다를 수 있으며 모집공지시 자세한 교육언어 참조
- 언어/레벨별 적정인원 모집시 개설되며 모집인원 미달시에는 개설되지 않음

3. 교육대상
- 대학(원)생

In [ ]:
# 각 패턴에 대해 검열 수행
test = re.sub(tel_number, '', test)
test = re.sub(phone_number, '', test)
test = re.sub(fax_number, '', test)
test = re.sub(address,'',test)
test = re.sub(email_pattern,'',test)
test = re.sub(link_http,'',test)
test = re.sub(link_www,'',test)
test = re.sub(link_hufs,'',test)
test = re.sub(punctuation_pattern,'',test)
print(test)

In [ ]:
def preprocess_text(text):
    # 링크 제거 (http:// 또는 https:// 로 시작하는 링크)
    text = re.sub(r'https?://\S+', '', text)
    # Zoom 회의 참가 ID 제거 _ 작동안함
    #text = re.sub(r'Zoom 회의 참가 ID \d{3} \d{3} \d{4}', '', text)
    # 전화번호 제거 (괄호 선택적으로)
    text = re.sub(r'\(?0\d{2,3}\)?[- ]?\d{3,4}[- ]?\d{4}', '', text)
    # 전번 제거
    text = re.sub(r'\d{2}-\d{4}-\d{4}', '', text)
    # 팩스번호 제거
    text = re.sub(r'FAX\.\d{2}-\d{3}-\d{4}', '', text)
    # 주소 제거
    text = re.sub(r'서울시 동대문구 이문로 107', '', text)
    # 이메일 제거
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    # www 링크 제거
    text = re.sub(r'\bwww\.\S*', '', text)
    # hufs.ac.kr 링크 제거
    text = re.sub(r'hufs\.ac\.kr', '', text)
    # 문장부호 제거
    text = re.sub(r'[,.:!?\\|※‘’”“→-○☏『』]', '', text)

    return text


# 'content' 열에 전처리 함수를 적용하여 새로운 열 생성
gongji2_copy['cleaned_content'] = gongji2_copy['content'].apply(preprocess_text)

NameError: name 're' is not defined

In [ ]:
print(gongji2_copy.loc[300, 'cleaned_content'])

In [ ]:
spacing = Spacing()

# 띄어쓰기 함수 적용
gongji2_copy['cleaned_content'] = gongji2_copy['cleaned_content'].apply(lambda x: spacing(x))

# 결과 확인
print( gongji2_copy.loc[300, 'cleaned_content'])
print( gongji2_copy.loc[300, 'cleaned_content'])


In [ ]:
# 결과 확인
print( gongji2_copy.loc[121, 'cleaned_content'])
print( gongji2_copy.loc[121, 'cleaned_content'])


제6회 한국외대 인문학 / 정보·기록학 화상 세미나 일 시 2020 11 1 일요일 오후 8시 발 표 박춘원 (위즈메타 CTO 한국외대 대학원 정보기록학과 겸임교수) 토 론 이해영 (명지대 기록정보과학전문대학원 교수) 정유진 (한국외대 컴퓨터 전자시스템공학부 교수) 유진양 (오썸피아 CTO) 장 소 줌 (zoom) 화상회의 시스템  url 주소는 상단 오픈채팅방에 화상 세미나 30분 전 공지됩니다 시작 15분 전인 7시 45분부터 입장 가능한 점 참고 부탁드립니다 2020 10 정보기록 학과
제6회 한국외대 인문학 / 정보·기록학 화상 세미나 일 시 2020 11 1 일요일 오후 8시 발 표 박춘원 (위즈메타 CTO 한국외대 대학원 정보기록학과 겸임교수) 토 론 이해영 (명지대 기록정보과학전문대학원 교수) 정유진 (한국외대 컴퓨터 전자시스템공학부 교수) 유진양 (오썸피아 CTO) 장 소 줌 (zoom) 화상회의 시스템  url 주소는 상단 오픈채팅방에 화상 세미나 30분 전 공지됩니다 시작 15분 전인 7시 45분부터 입장 가능한 점 참고 부탁드립니다 2020 10 정보기록 학과


### gongji2-> gongji_2_spacing.csv
- 기본 전처리 후 띄어쓰기 적용한 것 저장해둔 상태
- 전처리 좀 더 하고 품사태깅(okt)
- 빈도확인 및 사용자 사전 만들기
- TF_IDF 만들어보기

In [ ]:
# 결과를 CSV 파일로 저장
gongji2_copy.to_csv('gongji_2_spacing.csv', index=False)

# 저장된 파일 경로 확인
print("File saved asgongji_2_spacing.csv")

File saved asgongji_2_spacing.csv


In [ ]:
pwd

'/content'

In [ ]:
test = pd.read_csv('./drive/MyDrive/csv모음/gongji_2_spacing.csv')
test.head(10)

,index,date,title,content,cleaned_content
0,2127,2020-12-03,동계 현장실습(인턴십) 모집 공고(1차),\n동계 현장실습(인턴십) 모집 공고(1차)한국외국어대학교SW중심대학사업단(AI교육...,동계 현장실습(인턴십) 모집 공고(1차) 한국외국어대학교 SW중심대학 사업단(AI교...
1,2126,2020-12-03,HK+<접경인문학> 연구단 비대면 온라인학술대회 개최,\nHK+<접경인문학> 연구단 비대면 온라인학술대회 개최우리대학 역사문화연구소(소장...,HK+<접경인문학> 연구단 비대면 온라인 학술대회 개최우리대학 역사문화연구소(소장 ...
2,2125,2020-12-03,코로나19 관련 현황 및 조치사항(2020.12.03),\n코로나19 관련 현황 및 조치사항(2020.12.03)1. 부서별 주요 조치사항...,코로나 19 관련 현황 및 조치 사항(20201203) 1 부서별 주요 조치 사항(...
3,2124,2020-12-03,코로나19 발병현황 및 행동지침 (2020.12.03),\n코로나19 발병현황 및 행동지침 (2020.12.03)1.국내 신고 및 검사현황...,코로나 19 발병 현황 및 행동지침 (20201203) 1국내 신고 및 검사 현황(...
4,2123,2020-12-03,학생 핵심역량 통합관리시스템 네이밍 공모전,\n학생 핵심역량 통합관리시스템 네이밍 공모전교육선진화센터에서는우리 대학의핵심역량을...,학생 핵심역량 통합관리시스템 네이밍 공모전 교육선진화센터에서는 우리 대학의 핵심역량...
5,2122,2020-12-02,[공통] [Better Than Abroad] 20주 정규 주간과정 교육생 모집,\n [Better Than Abroad] 20주 정규 주간과정 교육생 모집\n20...,[Better Than A broad] 20주 정규 주간과 정 교육생 모집 20주 ...
6,2121,2020-12-02,코로나19 발병현황 및 행동지침 (2020.12.02),\n코로나19 발병현황 및 행동지침 (2020.12.02)1.국내신고및검사현황(20...,코로나 19 발병 현황 및 행동지침 (20201202) 1국내 신고 및 검사 현황(...
7,2120,2020-12-02,코로나19 관련 현황 및 조치사항(2020.12.02),\n코로나19 관련 현황 및 조치사항(2020.12.02)1. 부서별 주요 조치사항...,코로나 19 관련 현황 및 조치 사항(20201202) 1 부서별 주요 조치 사항(...
8,2119,2020-12-02,2021년도 정보통신 및 정보문화 유공 정부포상 계획 안내,\n2021년도 「정보통신 및 정보문화 유공」 정부포상 계획 공고관련 : 과학기술정...,2021년도 「정보통신 및 정보문화 유공」 정부포상 계획 공고 관련 과학기술정보통신...
9,2118,2020-12-01,"교육만족도조사 참여 안내 (학부모, 동문, 기업인사 대상)","\n2020학년도 한국외국어대학교 교육만족도조사 참여 안내 (학부모, 동문, 기업인...",2020학년도 한국외국어대학교 교육만족도 조사 참여 안내 (학부모 동문 기업인사 대...


In [ ]:
test.loc[5,'cleaned_content']

'[Better Than A broad] 20주 정규 주간과 정 교육생 모집 20주 주간과정은 말하기 듣기 읽기 쓰기를 종합적으로 학습할 수 있는 어학 집중 프로그램으로 유학 준비생 취업준비생 기관 및 기 업체 임직원 일반인 등을 대상으로 합니다 특히 과정 이 수 후 교육성과 만족도 면에서 높은 평가를 받아 프로그램의 우수성을 널리 인정받고 있습니다 #HUFS  #HUFSFLC #Bett erThanA broad #IMMERSION #ImmersionLearning #TheBestWay ToLearnF oreignLanguages #BTA 20주 정규 주간과 정 공지사항 20주 정규 주간과정은 소수(학급 당 최대 12명) 정예로 대면 수업 진행합니다 다만 코로나 19와 같은 감염증과 관련하여 부득이한 경우 화상수업으 로 전환되거나 일시적 휴강이 있을 수도 있습니다 그러한 경우 별도로 개별 공지 드리며 해당 내용을 반드시 숙지 후 접수 부탁드립니다 1 2021년 상반기 교육일정 - 접수 기간 20201214(월)~20210110(일) - 교육기간 20210201(월)~20210618(금) - 교육시간 월~금 0930~1620 1일 6시간 / 총 20주 - 반편성 시험 20210116(토) - 교육비 4130000원 * 600시간(5개월) 총 금액 교재비 별도 2 모집 언어 영어 중국어 - 반편성시험 결과에 따라 학급별 적정 인원 모집 시 개강합니다 - 상기 언어 외 기타언어는 별도 문의 바랍니다 3 교육 대상 - 대학(원)생 취업준비생 유학 준비생 기관 및 기 업체 임직원 일반인 등 제한 없음 - 미 성년자는 수강이 불가 합니다4 접수 방법 step 1 학사정보시스템 가입 및 로그인 step 2 수강신청 메뉴 클릭 step 3 희망하는 과정 수강신청 step 4 교육비 납입(카드결제 가능) *외국어 연수평가원 홈페이지 *학사정보시스템 메인화면 수강신청 메뉴 또는 학사정보시스템 메뉴 클릭 5 반편성시험(레벨테스트) - 시험 방식 필기시험 11 인터뷰 - 시험일시 

In [ ]:
#복사
pre_gongji2_copy = test.copy()
print(pre_gongji2_copy.loc[501,'cleaned_content'])
print("####################################")
print(pre_gongji2_copy.loc[501,'content'])

대면 수업 제한적 허용에 따른 국제학사 코로나 19 상황에 따라 2020학년도 1학기 기숙사 운영에 차질이 발생한 점 사생 여러분의 많은 양해 바랍니다 5월 11일(월)부터 대면 수업이 제한적으로 허용됨에 따라 기숙사비 환불 등 안내사항을 아래와 같이 공지합니다 1 기숙사비 환불가 현재 거주자 ( ~ 5월 10일 입사자) 일 단위 기준으로 환불 구분 환불비  고1 동계방학 이후 계속 거주 사생환불 금액 없음 계속 거주 2 3월 14일(토) ~ 3월 15일(일) 입사자 1주 환불 3 3월 16일(월) 이후 입사자 1주 + 3월 16일(월)부터 일 단위 환불*현재 거주자는 종합정보시스템 환불계좌 입력란에 본인 명의 계좌 입력나 퇴사 신청자 3월 ~ 5월 10일(일) 퇴사자 1) 가항에 따라 실제 거주했던 기간 기숙사비 를 제외하고 환불 2) 기존 환불 규정에 따라 이미 환불 받은 퇴사생은 가항의 금액에 추가하여 환불 예정 다 미 입사 취소자 - 2020-1학기 기숙사비를 완납하였으나 입사하지 않은 학생 중 5월 10일(일)까지 입사 취소 신청하는 경우 기숙사비 전액 환불라 환불 소요기간 약 2~3주 소요됨 단 입사 취소 문의 증가로 지연될 수 있음 위 규정은 5월 10일(일 )까지만 적용되는 기준 임 - 5월 11일(월)부터 아래 [국제학사 입사비 반환 내규]에 따라 환불함 (5월 10일(일)까지 취소 신청을 하지 않을 시 입사하지 않았더라도 입사한 것으로 간주하고 5월 11일(월)부터 퇴사일까지 해당 기숙사비가 계산됨) 참고 국제학사 입사비 반환 내규에 따른 환불금 5/14까지 퇴사 25% + 보증금 환불 - 5/1 5 ~ 6/13까지 퇴사 10% + 보증금 환불 - 6/14~ 퇴사 보증금 환불 2 기숙사 입사 수속가 미 입사 취소자 및 중도 퇴사자 재입사 가능나 입사를 원하는 사생은 국제학사 GlobeeD orm 홈페이지 자료실 11번 추가 입사신청서를 작성하여 국제학사 이메일()로 제출 2020 4 29국 제 학 사 장
#####################

### 0428~
- 정제된 텍스트 바탕으로 Okt뽑기시작

In [ ]:
#1. Konlpy에서 Okt를 불러옵니다.
!pip install konlpy
#2. konlpy 실행을 위한 자바어쩌구 다운
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#3. 필요한 설정
import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
from konlpy.tag import Okt

okt = Okt()
okt.morphs("한국외국어대학교에서 안내드립니다. 오늘은 학교수업이 없습니다.", stem = True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 49.0 MB/s eta 0:00:00


['한국',
 '외국어',
 '대학교',
 '에서',
 '안',
 '내',
 '드리다',
 '.',
 '오늘',
 '은',
 '학교',
 '수업',
 '이',
 '없다',
 '.']

In [ ]:
#복사
pre_gongji2_copy = test.copy()
print(pre_gongji2_copy.loc[501,'cleaned_content'])
print("####################################")
print(pre_gongji2_copy.loc[501,'content'])

In [ ]:
# 부동 소수점 값을 문자열로 변환하여 데이터프레임 열의 모든 값이 문자열로 구성되도록 함
pre_gongji2_copy['cleaned_content'] = pre_gongji2_copy['cleaned_content'].astype(str)

# 변환된 문자열 값을 사용하여 단어 리스트 생성
words = " ".join(pre_gongji2_copy['cleaned_content'].tolist())
words = okt.morphs(words, stem=True)
print(len(words))

1019070


In [ ]:
from collections import Counter
frequent = Counter(words).most_common()
frequent


In [ ]:
# 결과를 텍스트 파일로 저장
with open('word_frequency.txt', 'w', encoding='utf-8') as f:
    for word, count in frequent:
        f.write(f'{word}: {count}\n')
from google.colab import files
files.download('word_frequency.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 학사
- 공지 프로세스 이해를 위한 전처리 및 Tf-IDF까지 일단 가보기
- 전처리 리스트가 어느정도 완성될것이라 기대하며ㅠㅠ

In [ ]:
haksa_copy

,index,date,title,content
0,1,2020-01-06,[공통] 2019-2학기 이중전공 변경 배정 확정 공고,\n\n\n\n\n\n[공통]이중전공변경 배정 확정 공고\n \n2019학년도 제2...
1,2,2020-01-13,[공통] 2020학년도 1학기 수강신청 공고,\n2020학년도 1학기 수강신청 공고 2020학년도 제1학기 수강신청에 대하여 아...
2,3,2020-01-13,[공통] 2020학년도 전기 국제지역대학원 학석사연계과정 모집 안내,\n2020학년도 전기 국제지역대학원 학석사연계과정 모집 안내\n \n2020학년도...
3,4,2020-01-13,[공통] 2020학년도 전기 일반대학원 학석사연계과정 모집 안내,\n2020학년도 전기 일반대학원 학석사연계과정 모집 안내\n\n2020학년도 전기...
4,5,2020-01-15,[글로벌]졸업 예비사정 결과 조회,\n[글로벌] 졸업 예비사정 결과 조회글로벌캠퍼스 졸업(2020년 2월) 예비사정 ...
...,...,...,...,...
617,618,2024-03-13,[공통] 생리 유고 신청 안내,\n\n\n\n\n\n생리 유고 신청 안내\n\n\n\n\n\n 생리로 인한 유고...
618,619,2024-03-13,[공통] 졸업예정자(해당학기 졸업가능자)의 조기취업 유고결석 신청 안내,\n\n\n\n\n\n졸업예정자(해당학기 졸업가능자)의 조기취업유고결석 신청 안내\...
619,621,2024-03-15,[글로벌]2024-1학기 폐강(2차) 공고(글로벌캠퍼스),\n\n\n\n\n\n2024-1학기 폐강(2차) 공고(글로벌캠퍼스)\n\n\n\n...
620,620,2024-03-15,[서울] 2024-1학기 폐강(2차) 공고(서울캠퍼스),\n\n\n\n\n\n2024-1학기 폐강(2차) 공고(서울캠퍼스)\n\n\n\n\...


In [ ]:
import re

### 학사 전처리 유의사항 적는 곳
- 이중전공 사전에 넣어줄 것: 이중전공을 잘 인식못함
- 이수도 넣어주기 (이/수로 인식함) _ 이수, 이수구분
- 미배정도 미/배정으로 끊김
- 학년도를 학년/도(조사)로 인식함

In [ ]:
from pykospacing import Spacing
spacing = Spacing()
kospacing_sent = spacing(haksa_copy.loc[0,'content'])
print(haksa_copy.loc[0,'content'])
print(kospacing_sent)







[공통]이중전공변경 배정 확정 공고
 
2019학년도 제2학기 이중전공 변경 배정 결과를 아래와 같이 공고함.
 
아 래
 
1. 결과조회 : 종합정보시스템 → 전공/교직관리 → 이중전공 변경신청 → 안내사항
 
2. 이중전공 변경 시에 배정된 학생은 추후 이중전공 변경 신청 불가
 미배정된 학생은 기존 이수중인 이중전공/부전공이 유지되며, 추후 이중전공 변경 기간 중 신청 가능
 
3. 2020년 2월 졸업예정자(4학년 수료_졸업대기 포함)는 반드시 1월초까지 이수구분변경신청을 하여 
 영역별 졸업 학점에 누락 되는 일이 없도록 해야 함
 
4. 이중전공 변경에 따라 성적 조회 후, 변경되지 않은 이수구분의 수정은 학사종합지원센터 내 
 1전공 소속 단과대학 창구에 방문하여 이수구분변경신청원을 작성하여 제출 하여야 함(방학 및 학기 중 가능)
 
5. 기타사항
(1) 부전공, 전공심화+부전공, 단일전공(기존 이중전공/부전공 취소) 신청기간
 2020.1.29~1.31 종합정보시스템에서 신청(1월 중순 학사공지 참조 바람)
(2) 다음 학기 이중전공 변경기간은 5월초로 예정되어 있음.
 
2020. 1
 
교무처장

[공통] 이중 전 공 변경 배정 확정 공고 2019학년도 제2학기 이 중전공 변경 배정 결과를 아래와 같이 공고함. 아 래 1. 결과 조회 : 종합정보시스템 → 전공/교직 관리 → 이 중전공 변경신청 → 안내사항 2. 이중전공 변경 시에 배정된 학생은 추후 이 중전공 변경 신청 불가 미 배정된 학생은 기존 이 수중인 이 중전공/부전공이 유지되며,  추후 이 중전공 변경 기간 중 신청 가능 3. 2020년 2월 졸업예정자(4학년 수료 _ 졸업대기 포함)는 반드시 1월 초까지 이 수구분 변경 신청을 하여 영역별 졸업 학점에 누락 되는 일이 없도록 해야 함 4. 이중전공 변경에 따라 성적 조회 후, 변경되지 않은 이 수구분의 수정은 학사종합지원센터 내 1전공 소속 단과대학 창구에 방문하여 이수 구분변경신 청원을 작성하여 제출 하여야 함(방학 

In [ ]:
def preprocess_text(text):
    # 링크 제거 (http:// 또는 https:// 로 시작하는 링크)
    text = re.sub(r'https?://\S+', '', text)
    # Zoom 회의 참가 ID 제거 _ 작동안함
    #text = re.sub(r'Zoom 회의 참가 ID \d{3} \d{3} \d{4}', '', text)
    # 전화번호 제거 (괄호 선택적으로)
    text = re.sub(r'\(?0\d{2,3}\)?[- ]?\d{3,4}[- ]?\d{4}', '', text)
    # 전번 제거
    text = re.sub(r'\d{2}-\d{4}-\d{4}', '', text)
    # 팩스번호 제거
    text = re.sub(r'FAX\.\d{2}-\d{3}-\d{4}', '', text)
    # 주소 제거
    text = re.sub(r'서울시 동대문구 이문로 107', '', text)
    # 이메일 제거
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    # www 링크 제거
    text = re.sub(r'\bwww\.\S*', '', text)
    # hufs.ac.kr 링크 제거
    text = re.sub(r'hufs\.ac\.kr', '', text)
    #마지막 날짜 양식 변형
    #text = re.sub(r'\d{4}\.\s\d{2}\.\s\d{2}\.', '', text)
    #마지막 날짜
    #text = re.sub(r'\d{4}\.\s\d{2}\.', '', text) #과제 제출등에 유의
    # 문장부호 제거
    text = re.sub(r'[,.:!?\\|※‘’”“→-○☏『』)(]', '', text)
    #학사한정: 교무처장 제거
    text = re.sub(r'교무처장', '', text)
    text = re.sub(r'\n','',text)

    return text


# 'content' 열에 전처리 함수를 적용하여 새로운 열 생성
haksa_copy['cleaned_content'] = haksa_copy['content'].apply(preprocess_text)
haksa_copy.tail()

,index,date,title,content,cleaned_content
617,618,2024-03-13,[공통] 생리 유고 신청 안내,\n\n\n\n\n\n생리 유고 신청 안내\n\n\n\n\n\n 생리로 인한 유고...,생리 유고 신청 안내 생리로 인한 유고결석 신청 방법을 아래와 같이 안내합니다- ...
618,619,2024-03-13,[공통] 졸업예정자(해당학기 졸업가능자)의 조기취업 유고결석 신청 안내,\n\n\n\n\n\n졸업예정자(해당학기 졸업가능자)의 조기취업유고결석 신청 안내\...,졸업예정자해당학기 졸업가능자의 조기취업유고결석 신청 안내졸업예정자해당학기 졸업가능자...
619,621,2024-03-15,[글로벌]2024-1학기 폐강(2차) 공고(글로벌캠퍼스),\n\n\n\n\n\n2024-1학기 폐강(2차) 공고(글로벌캠퍼스)\n\n\n\n...,2024-1학기 폐강2차 공고글로벌캠퍼스2024-1학기 글로벌캠퍼스 폐강2차 강좌를...
620,620,2024-03-15,[서울] 2024-1학기 폐강(2차) 공고(서울캠퍼스),\n\n\n\n\n\n2024-1학기 폐강(2차) 공고(서울캠퍼스)\n\n\n\n\...,2024-1학기 폐강2차 공고서울캠퍼스2024학년도 제1학기 서울캠퍼스 폐강2차 강...
621,622,2024-03-26,[공통]2024-여름계절학기 개설희망 교과목 수요조사 실시 공고,\n계절학기 개설희망 교과목 수요조사 실시 공고\n\n2024학년도여름계절학기 개설...,계절학기 개설희망 교과목 수요조사 실시 공고2024학년도여름계절학기 개설희망 교과목...


In [ ]:
print(haksa_copy.loc[118, 'cleaned_content'])
print("________________________________________")
print(haksa_copy.loc[118, 'content'])

2020학년도 제 2학기 브릭스 융합전공 졸업논문 제출 일정 공고  2020학년도 제 2학기 브릭스 융합전공 졸업논문 제출 일정을 아래와 같이 공고함  - 아 래 -   1 대상  현재 3학년 과정 이수 후 7학기 이상 재적생 중 브릭스 융합전공을 이중전공  또는 제 2전공으로 이수하는 학생   2 일정  가 논문 작성계획서 제출 마감  2020 11 04 수  나 논문 제출 마감  2020 12 04 금   * 졸업논문 제출 시 첨부된 '논문제출 청구 및 승인서' 양식과 '졸업논문 표지' 양식을 작성하여 함께 제출지도교수란은 공란으로 제출   3 분량  A4 용지 20매 이상 글씨 크기 11 포인트 줄간격 160% 200자 원고지 100매 이상과 동일   4 제출장소  소속 캠퍼스 BRICs주관 각 학과서울/글로벌 5브릭스전공 연락처 조교 박 지 상2020 10 12BRICs 융합전공 주임교수
________________________________________




2020학년도 제 2학기 브릭스 융합전공 졸업논문 제출 일정 공고




  2020학년도 제 2학기 브릭스 융합전공 졸업논문 제출 일정을 아래와 같이 공고함. 

 
- 아 래 - 

 
 1. 대상 : 현재 3학년 과정 이수 후 7학기 이상 재적생 중, 브릭스 융합전공을 이중전공 
 또는 제 2전공으로 이수하는 학생
 
  2. 일정 
 가. 논문 작성계획서 제출 마감 : 2020. 11. 04. (수) 
 나. 논문 제출 마감 : 2020. 12. 04. (금) 
  * 졸업논문 제출 시 첨부된 '논문제출 청구 및 승인서' 양식과 '졸업논문 표지' 양식을
 작성하여 함께 제출(지도교수란은 공란으로 제출)
 
  3. 분량 : A4 용지 20매 이상, 글씨 크기 11 포인트, 줄간격 160%
 (200자 원고지 100매 이상과 동일)
 
  4. 제출장소 : 소속 캠퍼스 BRICs주관 각 학과(서울/글로벌)

 5.브릭스전공 연락처: 조교 박 지 상(hufsbrics@naver.com)

In [ ]:
# kospacing 안하고 okt 넣어보기
#1. Konlpy에서 Okt를 불러옵니다.
!pip install konlpy
#2. konlpy 실행을 위한 자바어쩌구 다운
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#3. 필요한 설정
import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
from konlpy.tag import Okt

okt = Okt()
okt.morphs("한국외국어대학교에서 안내드립니다. 오늘은 학교수업이 없습니다.", stem = True)

['한국',
 '외국어',
 '대학교',
 '에서',
 '안',
 '내',
 '드리다',
 '.',
 '오늘',
 '은',
 '학교',
 '수업',
 '이',
 '없다',
 '.']

In [ ]:
#빈도확인용
okt = Okt()
words = " ".join( haksa_copy['cleaned_content'].tolist())
words = okt.morphs(words,stem=True)

In [ ]:
#품사태그로 쪼개주기
okt = Okt()
tagged_words = []
for sentence in haksa_copy['cleaned_content']:
    tagged_sentence = okt.pos(sentence, stem=True)
    tagged_words.append(tagged_sentence)

tagged_words

In [ ]:
#품사 없이 단어만
okt = Okt()
words_list = []

for sentence in haksa_copy['cleaned_content']:
    words = okt.morphs(sentence, stem=True)
    words_list.append(words)


In [ ]:
#태그된 단어 일단 메모장에넣어주기
from google.colab import files

with open('haksa_tagged_words.txt', 'w', encoding='utf-8') as file:
    for sentence in tagged_words:
        line = ' '.join([f'{word}/{tag}' for word, tag in sentence])
        file.write(line + '\n')

files.download('haksa_tagged_words.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#품사없이 단어만 일단 메모장에넣어주기
from google.colab import files

# 파일을 쓰기 모드로 열고 데이터를 쓴 후 파일을 다운로드합니다.
with open('haksa_words_list.txt', 'w', encoding='utf-8') as file:
    for words in words_list:
        line = ' '.join(words)
        file.write(line + '\n')

# 다운로드할 파일을 지정합니다.
files.download('haksa_words_list.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
words[4]

'전공'

In [ ]:
from collections import Counter
frequent = Counter(words).most_common()
frequent

[('전공', 5341),
 ('하다', 5260),
 ('1', 3667),
 ('신청', 3647),
 ('2', 3400),
 ('에', 3305),
 ('-', 3214),
 ('을', 3171),
 ('학기', 3030),
 ('대학', 2706),
 ('은', 2682),
 ('과', 2663),
 ('이', 2653),
 ('수', 2472),
 ('의', 2449),
 ('학과', 2394),
 ('/', 2303),
 ('및', 2267),
 ('되다', 2056),
 ('캠퍼스', 1976),
 ('3', 1942),
 ('수강', 1926),
 ('학생', 1893),
 ('수강신청', 1801),
 ('학점', 1789),
 ('졸업', 1600),
 ('4', 1530),
 ('~', 1519),
 ('글로벌', 1480),
 ('경우', 1477),
 ('이중', 1467),
 ('학년', 1453),
 ('으로', 1453),
 ('함', 1445),
 ('한', 1423),
 ('변경', 1404),
 ('기간', 1403),
 ('를', 1344),
 ('학부', 1314),
 ('자', 1269),
 ('에서', 1259),
 ('서울', 1215),
 ('이수', 1197),
 ('6', 1150),
 ('후', 1145),
 ('불가', 1103),
 ('가능', 1103),
 ('제출', 1098),
 ('는', 1082),
 ('확인', 1064),
 ('통', 1047),
 ('금', 1013),
 ('도', 1007),
 ('시', 1003),
 ('납부', 997),
 ('과목', 992),
 ('중', 986),
 ('가', 978),
 ('5', 965),
 ('수업', 963),
 ('부', 945),
 ('7', 904),
 ('내', 900),
 ('없다', 891),
 ('로', 887),
 ('취소', 879),
 ('성적', 865),
 ('학사', 859),
 ('배정', 850),
 ('학교', 8

### tf-idf 참고링크
https://heytech.tistory.com/337


In [ ]:
#tf-idf 넣어줄 사이킷런다운
!pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 71.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
#판다스로열어줌
documents = pd.read_csv("haksa_words_list.txt")
documents

In [ ]:
#공지 읽어오기
with open("haksa_words_list.txt",'r',encoding = 'utf-8') as file:
  lines = file.readlines()
  data = [line.strip() for line in lines]
documents = data
documents[1]

'2020 학년 도 1 학기 수강신청 공고 2020 학년 도 제 1 학기 수강신청 에 대하 여 아래 와 같이 공고 하다 - 아 래 - 1 강의 시간표 · 강의 계획 서 조회 및 수강 편람 열람 가다 강의 시간표 및 강의 계획 서 조회 학교 홈페이지 Hot Link < 강의 시간표 조회 > 변동 사항 이 실시간 으로 반영 되다 수강신청 전 다시 확인 바람나다 수강 편람 열람 학교 홈페이지 Hot Link < 수강 편람 > 수강 편람 은 매 학기 수강신청 전 학교 홈페이지 Hot Link 에 게시 1월 말 예정 2 수강신청 일정 2019-2 학기 부터 장바구니 명칭 변경 예비 수강신청 하다 분일 정비 고 예비 수강신청 하다 장바구니 2020 1 29 수 1000 ~ 수강신청 변경 종료 시 까지 학교 홈페이지 Link 수강신청 / 예비 수강신청 함 바로가기 수강신청 재학생 4 학년 2020 2 3월 1000 ~ 15003 학년 2020 2 4 화 1000 ~ 15002 학년 2020 2 5 수 1000 ~ 15001 학년 2020 2 6 목 1000 ~ 1500 전체 학년 2020 2 7 금 1000 ~ 1500 신 · 편입 생 2020 3 9월 1000 ~ 1600 수강신청 변경 2020 3 16월 ~ 3 20 금 매일 1000 ~ 1600 가다 본인 의 1 전공 학년 기준 수 강일 자 에 1 전공 및 이 중부 전공 수강신청 나 수강신청 학년 수강신청 전 예비 수강신청 함 에 로그인 하다 필히 사전 확인 2020-1 학기 등록 횟수 기준 4 학년 등록 7회 이상 3 학년 등록 5~6회 2 학년 등록 3~4회 1 학년 등록 1~2회 다 기타 알다 편입 생 및 외국인 교환학생 외국인 방문 학생 입학 한 첫 학기 재수강 신청 불가 2020-1 학기 복학 예정자 복학 신청 전 수강신청 기간 에 수강신청 가능 2020-1 학기 휴학 신청 자 수강신청 전 휴학 신청 수강신청 로그인 불가 수강신청 후 휴학 신청 휴학 승인 시 수강신청 과목 일괄 삭제 되다 수강신청 기간 [ 2

In [ ]:
vectorizer = CountVectorizer()
# Document Term Matrix
dtm = vectorizer.fit_transform(documents)

# Term Freqeuncy
tf = pd.DataFrame(dtm.toarray(), columns=vectorizer.get_feature_names_out())
# Document Frequency
df = tf.astype(bool).sum(axis = 0)
# 문서 개수
D = len(tf)

# Inverse Document Frequency
idf = np.log((D) / (df+1))
# TF-IDF
tfidf = tf * idf
tfidf = tfidf / np.linalg.norm(tfidf, axis = 1, keepdims = True)
tfidf

,000,00101106,00101107,00101109,00101113,00101117,00101118,00101119,00101120,00102112,...,흐름,흠화,희곡,희다,희망,희문목,희수,희화,히스토리,힌디어
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.011814,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.103092,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
618,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [ ]:
tf

,000,00101106,00101107,00101109,00101113,00101117,00101118,00101119,00101120,00102112,...,흐름,흠화,희곡,희다,희망,희문목,희수,희화,히스토리,힌디어
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
618,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
619,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
620,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df

000         18
00101106     1
00101107     1
00101109     1
00101113     1
            ..
희문목          1
희수           1
희화           2
히스토리         1
힌디어         10
Length: 8782, dtype: int64

In [ ]:
D

622

In [ ]:
tfidf

,000,00101106,00101107,00101109,00101113,00101117,00101118,00101119,00101120,00102112,...,흐름,흠화,희곡,희다,희망,희문목,희수,희화,히스토리,힌디어
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.011814,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.103092,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
618,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


# 코드 정리지점
### 다시 공지 전처리 해보고 tf-idf 해보기

In [1]:
#호출코드모음
import pandas as pd
import os
#마운트
from google.colab import drive
drive.mount('/content/drive')
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
#단어 빈도셀 때
from collections import Counter

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#kospacing 다운 및 호출
#!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
from pykospacing import Spacing
spacing = Spacing()
#kospacing_sent = spacing(new_sent)

In [3]:
#konlpy 다운 및 호출
#1. Konlpy에서 Okt
!pip install konlpy
#2. konlpy 실행을 위한 자바어쩌구 다운
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#3. 필요한 설정
import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
from konlpy.tag import Okt

okt = Okt()
okt.morphs("한국외국어대학교에서 안내드립니다. 오늘은 학교수업이 없습니다.", stem = True)


['한국외국어대학교', '에서', '안내드립니다', '.', '오늘', '은', '학교', '수업', '이', '없다', '.']

In [4]:
okt.morphs("안내 합니다")
okt.morphs("안내 드립니다")
okt.morphs("안내드립니다")

['안내드립니다']

In [5]:
#각종 문서 저장
gongji = pd.read_csv('./drive/MyDrive/csv모음/merge_gongji.csv')
haksa = pd.read_csv('./drive/MyDrive/csv모음/merged_haksa.csv')
hiring = pd.read_csv('./drive/MyDrive/csv모음/merged_hire.csv')
janghak = pd.read_csv('./drive/MyDrive/csv모음/merged_janghak.csv')
#공지가 너무 커 세개로 나눔
gongji_1 = pd.read_csv('./drive/MyDrive/csv모음/gongji_1.csv')
gongji_2 = pd.read_csv('./drive/MyDrive/csv모음/gongji_2.csv', encoding = "cp949")
gongji_3 = pd.read_csv('./drive/MyDrive/csv모음/gongji_3.csv')
gongji.head(5)

,index,date,title,content
0,1,2018-01-02,2018년 상반기 20주 주간/야간 외국어교육과정 교육생 모집,\n2018년 상반기 \n20주 \n주간/야간 외국어교육과정 교육생 \n모집기초부터...
1,2,2018-01-02,2018년 상반기 FLC 외국어교육과정 교육생 모집,\n2018년 상반기 FLC \n외국어교육과정 교육생 모집기초부터 고급까지! 해외연...
2,3,2018-01-02,2018년 상반기 10주 주말(토) 외국어교육과정 교육생 모집,\n2018년 상반기 \n10주 \n주말(토) 외국어교육과정 교육생 \n모집기초부터...
3,4,2018-01-03,대원빌딩 철거공사 입찰 관련 질의 응답,\n대원빌딩 철거공사 입찰 관련 질의 응답대원빌딩 철거공사 입찰과 관련한 질의사항에...
4,7,2018-01-04,2018학년도 제1학기 서울캠퍼스 국제학사 기숙사 신입생 모집 공고,\n2018학년도 제1학기 서울캠퍼스 국제학사 기숙사 신입생 모집 공고\n2018학...


In [6]:
print(len(gongji))
print(len(gongji_1))

6713
6713


In [5]:
#파일복사
# 원본 데이터 보존 위해 copy 데이터 생성
gongji_copy = gongji.copy()
haksa_copy = haksa.copy()
hiring_copy = hiring.copy()
janghak_copy = janghak.copy()


gongji1_copy = gongji_1.copy()
gongji2_copy = gongji_2.copy()
gongji3_copy = gongji_3.copy()

In [6]:
#전처리 함수(통일)
def preprocess_text(text):
    # 링크 제거 (http:// 또는 https:// 로 시작하는 링크)
    text = re.sub(r'https?://\S+', '', text)
    # Zoom 회의 참가 ID 제거 _ 작동안함
    #text = re.sub(r'Zoom 회의 참가 ID \d{3} \d{3} \d{4}', '', text)
    # 전화번호 제거 (괄호 선택적으로)
    text = re.sub(r'\(?0\d{2,3}\)?[- ]?\d{3,4}[- ]?\d{4}', '', text)
    # 전번 제거
    text = re.sub(r'\d{2}-\d{4}-\d{4}', '', text)
    # 팩스번호 제거
    text = re.sub(r'FAX\.\d{2}-\d{3}-\d{4}', '', text)
    # 주소 제거
    text = re.sub(r'서울시 동대문구 이문로 107', '', text)
    # 이메일 제거
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    # www 링크 제거
    text = re.sub(r'\bwww\.\S*', '', text)
    # hufs.ac.kr 링크 제거
    text = re.sub(r'hufs\.ac\.kr', '', text)
    # 문장부호 제거
    text = re.sub(r'[,.:!?\\|※‘’”“→-○☏『』]', '', text)

    return text


# 'content' 열에 전처리 함수를 적용하여 새로운 열 생성
#gongji2_copy['cleaned_content'] = gongji2_copy['content'].apply(preprocess_text)

######################공통 구간#######################################

이 아래로 공지 병합 코딩진행

In [4]:
gongji2_copy['title']

NameError: name 'gongji2_copy' is not defined

In [9]:
#코로나 개수확인
corona_count = len(gongji2_copy[gongji2_copy['title'].str.contains('코로나19')])
# 결과 출력
print("코로나19가 포함된 데이터 개수:", corona_count)
print("삭제 전 데이터 개수:", len(gongji2_copy))

코로나19가 포함된 데이터 개수: 265
삭제 전 데이터 개수: 1766


In [10]:
#공지2 코로나 행 삭제
drop_covid_2 = gongji2_copy[gongji2_copy['title'].str.contains('코로나19')]
gongji2_copy.drop(drop_covid_2.index, inplace=True)
print("삭제 후 데이터 개수:", len(gongji2_copy))

삭제 후 데이터 개수: 1501


In [11]:
#공지1 코로나 행 삭제
#코로나 개수확인
corona_count = len(gongji1_copy[gongji1_copy['title'].str.contains('코로나19')])
# 결과 출력
print("코로나19가 포함된 데이터 개수:", corona_count)
print("삭제 전 데이터 개수:", len(gongji1_copy))

drop_covid_1 = gongji1_copy[gongji1_copy['title'].str.contains('코로나19')]
gongji1_copy.drop(drop_covid_1.index, inplace=True)
print("삭제 후 데이터 개수:", len(gongji1_copy))

코로나19가 포함된 데이터 개수: 1277
삭제 전 데이터 개수: 6713
삭제 후 데이터 개수: 5436


In [14]:
#'content' 열에 전처리 함수를 적용하여 새로운 열 생성
gongji2_copy['cleaned_content'] = gongji2_copy['content'].apply(preprocess_text)
gongji2_copy.tail()

,index,date,title,content,cleaned_content
1761,366,2018-06-26,[서울] 국제학사GlobeeDorm 기숙사생 우선 선발 대상자 변경 공고,\n국제학사GlobeeDorm 기숙사생 우선 선발 대상자 변경 공고 국제학사 사생...,\n국제학사GlobeeDorm 기숙사생 우선 선발 대상자 변경 공고 국제학사 사생...
1762,365,2018-06-26,제 11회 국제청년학자중문학술회의 및 제 8회 인문사회연구포럼 개최,\n제 11회 국제청년학자중문학술회의 및 제 8회 인문사회연구포럼 개최우리대학 대만...,\n제 11회 국제청년학자중문학술회의 및 제 8회 인문사회연구포럼 개최우리대학 대만...
1763,364,2018-06-26,[CELPE-BRAS] 2018학년도 Celpe-Bras 포르투갈어인증시험 응시 안내,2018학년도 Celpe-Bras \n포르투갈어인증시험 응시 안내브라질 교육부가 출...,2018학년도 Celpe-Bras \n포르투갈어인증시험 응시 안내브라질 교육부가 출...
1764,363,2018-06-25,[CORE] 공개강의(OCW) 개설 지원 공고,\n대학 인문역량 강화사업(CORE)공개강의(OCW) 개설 지원 공고(2018. 6...,\n대학 인문역량 강화사업(CORE)공개강의(OCW) 개설 지원 공고(2018 6 ...
1765,362,2018-06-22,[서울] 2018년 7월 심폐소생술 교육일정안내,\n2018년 7월 심폐소생술 교육일정안내 \n\n서울캠퍼스 보건실에서는 클리닉 프...,\n2018년 7월 심폐소생술 교육일정안내 \n\n서울캠퍼스 보건실에서는 클리닉 프...


In [ ]:
#빈도확인용
okt = Okt()
words = " ".join( gongji2_copy['cleaned_content'].tolist())
words = okt.morphs(words,stem=True)
words

In [ ]:
frequent = Counter(words).most_common()
frequent

In [3]:
pwd

'/content'

# 사용자 사전 고치기
- 아래의 절차 따르고 맨 아래 data write 쪽 정정해주기
- - 사용자 사전 만들기 (colab)
https://sirzzang.github.io/ai/AI-konlpy-userdic/

In [8]:
#사용자 사전 찾아가기
os.chdir('/usr/local/lib/python3.10/dist-packages/konlpy/java')
os.getcwd()

'/usr/local/lib/python3.10/dist-packages/konlpy/java'

In [9]:
#os.makedirs('./temp_fix')
os.chdir('./temp_fix')
os.getcwd()

'/usr/local/lib/python3.10/dist-packages/konlpy/java/temp_fix'

In [10]:
!jar xvf ../open-korean-text-2.1.0.jar

  created: META-INF/
 inflated: META-INF/MANIFEST.MF
  created: META-INF/maven/
  created: META-INF/maven/org.openkoreantext/
  created: META-INF/maven/org.openkoreantext/open-korean-text/
 inflated: META-INF/maven/org.openkoreantext/open-korean-text/pom.properties
 inflated: META-INF/maven/org.openkoreantext/open-korean-text/pom.xml
  created: org/
  created: org/openkoreantext/
  created: org/openkoreantext/processor/
  created: org/openkoreantext/processor/stemmer/
 inflated: org/openkoreantext/processor/stemmer/KoreanStemmer$.class
 inflated: org/openkoreantext/processor/stemmer/KoreanStemmer.class
  created: org/openkoreantext/processor/normalizer/
 inflated: org/openkoreantext/processor/normalizer/KoreanNormalizer.class
 inflated: org/openkoreantext/processor/normalizer/KoreanNormalizer$.class
 inflated: org/openkoreantext/processor/normalizer/KoreanNormalizer$Segment$.class
 inflated: org/openkoreantext/processor/normalizer/KoreanNormalizer$Segment.class
 inflated: org/openkorea

In [11]:
pwd

'/usr/local/lib/python3.10/dist-packages/konlpy/java/temp_fix'

In [12]:
# 사용자 사전 열기
with open(f"/usr/local/lib/python3.10/dist-packages/konlpy/java/temp_fix/org/openkoreantext/processor/util/noun/names.txt") as f:
    data = f.read()
data

'가몽\n가온\n갓세븐\n강새이\n게임닉가\n관우\n귀여미\n규\n김유이\n김준면\n까까런\n노컷\n누너예\n니노\n다마고치\n다이무스\n대학생\n데이브\n도요토미\n동운\n동이\n두주니\n디시인사이드\n디오\n라몹\n라스\n라옵\n멍구\n메이든\n명덕\n명량\n문민정부\n미네\n방엘리\n병헌\n붓다\n비정상회담\n빼빼로\n삼풍\n샤인온미\n성식\n성열\n세라문\n세라복\n세종대왕\n손권\n손책\n쇼미더머니\n쇼챔\n순규\n스라소니\n신동아\n신쓰패밀리\n신아라\n아베\n안상홍\n안홍준\n여누\n여랑\n여포\n연합\n오꾸닭\n요섭\n웃찾사\n원식\n유병언\n유비\n유이\n윤기형\n이나단\n이명박\n이완용\n임창용\n자괴\n자니윤\n자대련\n자유\n재중이\n전교조\n정윤회\n제갈량\n조자룡\n조조\n준면\n지오디\n지존파\n진영오\n차작가\n차트\n창섭\n챠트\n첸\n코르사주\n하무열\n하용파쿠\n혁재\n현이\n현태\n혜미\n김재경자경\n송이레만세\n한국외국어대학교\n김재경자경\n송이레만세\n한국외국어대학교\n김재경자경\n송이레만세\n한국외국어대학교\n김재경자경\n송이레만세\n한국외국어대학교\n김재경자경\n송이레만세\n한국외국어대학교\n안내\n'

In [13]:
data +='안내드립니다\n'
# 파일 새롭게 저장
with open("/usr/local/lib/python3.10/dist-packages/konlpy/java/temp_fix/org/openkoreantext/processor/util/noun/names.txt", 'w') as f:
    f.write(data)
data

'가몽\n가온\n갓세븐\n강새이\n게임닉가\n관우\n귀여미\n규\n김유이\n김준면\n까까런\n노컷\n누너예\n니노\n다마고치\n다이무스\n대학생\n데이브\n도요토미\n동운\n동이\n두주니\n디시인사이드\n디오\n라몹\n라스\n라옵\n멍구\n메이든\n명덕\n명량\n문민정부\n미네\n방엘리\n병헌\n붓다\n비정상회담\n빼빼로\n삼풍\n샤인온미\n성식\n성열\n세라문\n세라복\n세종대왕\n손권\n손책\n쇼미더머니\n쇼챔\n순규\n스라소니\n신동아\n신쓰패밀리\n신아라\n아베\n안상홍\n안홍준\n여누\n여랑\n여포\n연합\n오꾸닭\n요섭\n웃찾사\n원식\n유병언\n유비\n유이\n윤기형\n이나단\n이명박\n이완용\n임창용\n자괴\n자니윤\n자대련\n자유\n재중이\n전교조\n정윤회\n제갈량\n조자룡\n조조\n준면\n지오디\n지존파\n진영오\n차작가\n차트\n창섭\n챠트\n첸\n코르사주\n하무열\n하용파쿠\n혁재\n현이\n현태\n혜미\n김재경자경\n송이레만세\n한국외국어대학교\n김재경자경\n송이레만세\n한국외국어대학교\n김재경자경\n송이레만세\n한국외국어대학교\n김재경자경\n송이레만세\n한국외국어대학교\n김재경자경\n송이레만세\n한국외국어대학교\n안내\n안내드립니다\n'

In [14]:
data

'가몽\n가온\n갓세븐\n강새이\n게임닉가\n관우\n귀여미\n규\n김유이\n김준면\n까까런\n노컷\n누너예\n니노\n다마고치\n다이무스\n대학생\n데이브\n도요토미\n동운\n동이\n두주니\n디시인사이드\n디오\n라몹\n라스\n라옵\n멍구\n메이든\n명덕\n명량\n문민정부\n미네\n방엘리\n병헌\n붓다\n비정상회담\n빼빼로\n삼풍\n샤인온미\n성식\n성열\n세라문\n세라복\n세종대왕\n손권\n손책\n쇼미더머니\n쇼챔\n순규\n스라소니\n신동아\n신쓰패밀리\n신아라\n아베\n안상홍\n안홍준\n여누\n여랑\n여포\n연합\n오꾸닭\n요섭\n웃찾사\n원식\n유병언\n유비\n유이\n윤기형\n이나단\n이명박\n이완용\n임창용\n자괴\n자니윤\n자대련\n자유\n재중이\n전교조\n정윤회\n제갈량\n조자룡\n조조\n준면\n지오디\n지존파\n진영오\n차작가\n차트\n창섭\n챠트\n첸\n코르사주\n하무열\n하용파쿠\n혁재\n현이\n현태\n혜미\n김재경자경\n송이레만세\n한국외국어대학교\n김재경자경\n송이레만세\n한국외국어대학교\n김재경자경\n송이레만세\n한국외국어대학교\n김재경자경\n송이레만세\n한국외국어대학교\n김재경자경\n송이레만세\n한국외국어대학교\n안내\n안내드립니다\n'

In [15]:
!jar cvf ../open-korean-text-2.1.0.jar *

added manifest
ignoring entry META-INF/
adding: META-INF/maven/(in = 0) (out= 0)(stored 0%)
adding: META-INF/maven/org.openkoreantext/(in = 0) (out= 0)(stored 0%)
adding: META-INF/maven/org.openkoreantext/open-korean-text/(in = 0) (out= 0)(stored 0%)
adding: META-INF/maven/org.openkoreantext/open-korean-text/pom.properties(in = 119) (out= 110)(deflated 7%)
adding: META-INF/maven/org.openkoreantext/open-korean-text/pom.xml(in = 9127) (out= 2208)(deflated 75%)
ignoring entry META-INF/MANIFEST.MF
adding: org/(in = 0) (out= 0)(stored 0%)
adding: org/openkoreantext/(in = 0) (out= 0)(stored 0%)
adding: org/openkoreantext/processor/(in = 0) (out= 0)(stored 0%)
adding: org/openkoreantext/processor/stemmer/(in = 0) (out= 0)(stored 0%)
adding: org/openkoreantext/processor/stemmer/KoreanStemmer$.class(in = 7044) (out= 2892)(deflated 58%)
adding: org/openkoreantext/processor/stemmer/KoreanStemmer.class(in = 1439) (out= 972)(deflated 32%)
adding: org/openkoreantext/processor/normalizer/(in = 0) (ou

In [18]:
import os
from konlpy.tag import Okt
new_dir = '/content'

# 작업 디렉토리 변경
os.chdir(new_dir)
okt = Okt()
print(okt.nouns('송이레만세')) # ['송이레만세']
print(okt.morphs('김재경자경')) # ['김재경자경']
print(okt.morphs("한국외국어대학교"))

['송이레만세']
['김재경자경']
['한국외국어대학교']


In [2]:
pwd

'/content'

# ---------------------------------------------------------- 사전 수정하는 법 --------------------------------------------------